# **INTRODUCTIONS**

---

The source code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Stored Procedures](#stored-procedures)**
3. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **TABLES**
---

### STAGINGS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE TABLE(S).....
DROP TABLE IF EXISTS [VWSSTAGE].[ECDC_VACCINATION_COVERAGES]
GO

CREATE TABLE [VWSSTAGE].[ECDC_VACCINATION_COVERAGES] (
	[ID] [INT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[YEAR_WEEK_ISO] [VARCHAR](100) NULL,
	[REPORTING_COUNTRY] [VARCHAR](100) NULL,
	[DENOMINATOR] [VARCHAR](100) NULL,
	[NUMBER_DOSES_RECEIVED] [VARCHAR](100) NULL,
	[NUMBER_DOSES_EXPORTED] [VARCHAR](100) NULL,
	[FIRST_DOSE] [VARCHAR](100) NULL,
	[FIRST_DOSE_REFUSED] [VARCHAR](100) NULL,
	[SECOND_DOSE] [VARCHAR](100) NULL,
	[DOSE_ADDITIONAL1] [VARCHAR](100) NULL,
	[UNKNOWN_DOSE] [VARCHAR](100) NULL,
	[REGION] [VARCHAR](100) NULL,
	[TARGET_GROUP] [VARCHAR](100) NULL,
	[VACCINE] [VARCHAR](100) NULL,
	[POPULATION] [VARCHAR](100) NULL
);
GO

### INTERMEDIATES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE INTER TABLE(S).....
DROP TABLE IF EXISTS [VWSINTER].[ECDC_VACCINATION_COVERAGES]
GO

CREATE TABLE [VWSINTER].[ECDC_VACCINATION_COVERAGES](
	[ID] [INT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[YEAR_WEEK_ISO] [VARCHAR](100) NULL,
	[REPORTING_COUNTRY] [VARCHAR](100) NULL,
	[DENOMINATOR] [INT] NULL,
	[NUMBER_DOSES_RECEIVED] [INT] NULL,
	[NUMBER_DOSES_EXPORTED] [INT] NULL,
	[FIRST_DOSE] [INT] NULL,
	[FIRST_DOSE_REFUSED] [INT] NULL,
	[SECOND_DOSE] [INT] NULL,
	[DOSE_ADDITIONAL1] [INT] NULL,
	[UNKNOWN_DOSE] [INT] NULL,
	[REGION] [VARCHAR](100) NULL,
	[TARGET_GROUP] VARCHAR(100) NULL,
	[VACCINE] [VARCHAR](100) NULL,
	[POPULATION] [INT] NULL
);
GO

### REPORTS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE REPORT TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSREPORT].[ECDC_VACCINATION_COVERAGES]'))
CREATE TABLE [VWSREPORT].[ECDC_VACCINATION_COVERAGES](
	[ID] [INT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[REPORTING_COUNTRY] [VARCHAR](100) NULL,
    [UPTAKE_ONE_VACCINE_DOSE] [DECIMAL](18, 1) NULL
);
GO

IF NOT EXISTS (
	SELECT 
		* 
	FROM SYS.INDEXES 
	WHERE [NAME]='NCIX_RC_UOVD_ECDC_VACCINATION_COVERAGES' 
		AND [OBJECT_ID] = OBJECT_ID(N'[VWSREPORT].[ECDC_VACCINATION_COVERAGES]')
)
CREATE NONCLUSTERED INDEX [NCIX_RC_UOVD_ECDC_VACCINATION_COVERAGES]
ON [VWSREPORT].[ECDC_VACCINATION_COVERAGES](
    [REPORTING_COUNTRY],
	[UPTAKE_ONE_VACCINE_DOSE]
);
GO

# **VIEWS**
---

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

CREATE OR ALTER VIEW [VWSREPORT].[V_PBI_ECDC_VACCINATION_COVERAGES] 
AS
  SELECT
    [REPORTING_COUNTRY],
    [UPTAKE_ONE_VACCINE_DOSE]
  FROM [VWSREPORT].[ECDC_VACCINATION_COVERAGES];
GO

# **STORE PROCEDURES**
---

### STAGING &rarr; INTERMEDIATE MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

-- 1) CREATE STORE PROCEDURE(S) STAGE -> INTER.....
CREATE OR ALTER PROCEDURE [DBO].[SP_ECDC_VACCINATION_COVERAGES_STAGE_TO_INTER]
AS
BEGIN
    INSERT INTO [VWSINTER].[ECDC_VACCINATION_COVERAGES] (
        [YEAR_WEEK_ISO],
        [REPORTING_COUNTRY],
        [DENOMINATOR],
        [NUMBER_DOSES_RECEIVED],
        [NUMBER_DOSES_EXPORTED],
        [FIRST_DOSE],
        [FIRST_DOSE_REFUSED],
        [SECOND_DOSE],
        [DOSE_ADDITIONAL1],
        [UNKNOWN_DOSE],
        [REGION],
        [TARGET_GROUP],
        [VACCINE],
        [POPULATION]
    )
    SELECT
        [YEAR_WEEK_ISO],
        [REPORTING_COUNTRY],
        CAST([DENOMINATOR] AS INT) AS [DENOMINATOR],
        CAST([NUMBER_DOSES_RECEIVED] AS INT) AS [NUMBER_DOSES_RECEIVED],
        CAST([NUMBER_DOSES_EXPORTED] AS INT) AS [NUMBER_DOSES_EXPORTED],
        CAST([FIRST_DOSE] AS INT) AS [FIRST_DOSE],
        CAST([FIRST_DOSE_REFUSED] AS INT) AS [FIRST_DOSE_REFUSED],
        CAST([SECOND_DOSE] AS INT) AS [SECOND_DOSE],
        CAST([DOSE_ADDITIONAL1] AS INT) AS [DOSE_ADDITIONAL1],
        CAST([UNKNOWN_DOSE] AS INT) AS [UNKNOWN_DOSE],
        [REGION],
        [TARGET_GROUP],
        [VACCINE],
        CAST ([POPULATION] AS INT) AS [POPULATION]
    FROM 
        [VWSSTAGE].[ECDC_VACCINATION_COVERAGES];

    TRUNCATE TABLE [VWSSTAGE].[ECDC_VACCINATION_COVERAGES];
END;
GO

### INTERMEDIATE &rarr; REPORT MAPPING

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

-- 1) CREATE STORE PROCEDURE(S) INTER -> REPORT.....
CREATE OR ALTER PROCEDURE [dbo].[SP_ECDC_VACCINATION_COVERAGES_INTER_TO_REPORT]
AS
BEGIN
    WITH BASE_CTE AS
    (
        SELECT 
            [YEAR_WEEK_ISO],
            [REPORTING_COUNTRY],
            [DENOMINATOR],
            [NUMBER_DOSES_RECEIVED],
            [NUMBER_DOSES_EXPORTED],
            [FIRST_DOSE],
            [FIRST_DOSE_REFUSED],
            [SECOND_DOSE],
            [DOSE_ADDITIONAL1],
            [UNKNOWN_DOSE],
            [REGION],
            [TARGET_GROUP],
            [VACCINE],
            [POPULATION]
        FROM [VWSINTER].[ECDC_VACCINATION_COVERAGES]
        WHERE UPPER([TARGET_GROUP]) IN ('ALL') 
            AND [REGION] = [REPORTING_COUNTRY]
    ),
    TOTAL_CTE AS (
        SELECT 
            [REPORTING_COUNTRY],
            [DENOMINATOR],
            SUM([FIRST_DOSE]) AS [SUM_FIRST_DOSE]
        FROM [BASE_CTE]
        GROUP BY [REPORTING_COUNTRY], [DENOMINATOR]
    )
    INSERT INTO [VWSREPORT].[ECDC_VACCINATION_COVERAGES] (
        [REPORTING_COUNTRY],
        [UPTAKE_ONE_VACCINE_DOSE]
    )
    SELECT 
        [REPORTING_COUNTRY],
        CAST((CAST([SUM_FIRST_DOSE] AS DECIMAL) / [DENOMINATOR]) * 100 AS decimal(18,1)) AS [UPTAKE_ONE_VACCINE_DOSE]
    FROM TOTAL_CTE

    TRUNCATE TABLE [VWSINTER].[ECDC_VACCINATION_COVERAGES];
END

# **DATATINO CONFIGURATIONS**
---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'ECDC_VACCINATION_COVERAGES',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @schedule = '0 */1 * * *', -- EVERY HOUR
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'https://opendata.ecdc.europa.eu/covid19/vaccine_tracker/csv/data.csv'
    WHEN 'acceptance' THEN 'https://opendata.ecdc.europa.eu/covid19/vaccine_tracker/csv/data.csv'
    ELSE 'https://opendata.ecdc.europa.eu/covid19/vaccine_tracker/csv/data.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'Web'
    WHEN 'acceptance' THEN 'Web'
    ELSE 'Web'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = CONCAT('SOURCE_', @workflow_name);
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'YearWeekISO|ReportingCountry|Denominator|NumberDosesReceived|NumberDosesExported|FirstDose|FirstDoseRefused|SecondDose|DoseAdditional1|UnknownDose|Region|TargetGroup|Vaccine|Population',
    @target_columns = 'YEAR_WEEK_ISO|REPORTING_COUNTRY|DENOMINATOR|NUMBER_DOSES_RECEIVED|NUMBER_DOSES_EXPORTED|FIRST_DOSE|FIRST_DOSE_REFUSED|SECOND_DOSE|DOSE_ADDITIONAL1|UNKNOWN_DOSE|REGION|TARGET_GROUP|VACCINE|POPULATION',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'Colon',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
SET @source = CONCAT('dbo.SP_', @workflow_name, '_STAGE_TO_INTER');
SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_INTER');
SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO REPORT.....
SET @source = CONCAT('dbo.SP_', @workflow_name, '_INTER_TO_REPORT');
SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_INTER_TO_REPORT');
SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM INTER TO REPORT');

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = CONCAT('PROCESS_', @workflow_name, '_TO_STAGE');
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO INTER......
SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');
SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_INTER');

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.3) UPSERT STORED PROCEDURE PROCESS(S): INTER TO REPORT......
SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_INTER_TO_REPORT');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM INTER TO REPORT');
SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_INTER_TO_REPORT');

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_STAGE_TO_INTER');
SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name, '_TO_STAGE');
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;

-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_INTER_TO_REPORT');
SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_INTER_TO_REPORT');
SET @dependency_process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO